# Get character data

> Use the PoE API to get character data.

In [2]:
#| default_exp get.characters

In [3]:
#| hide
from nbdev.showdoc import *

In [9]:
#| export
import numpy as np
import pandas as pd
import requests
from traceback import format_exc
from tinydb import TinyDB, Query
from tqdm import tqdm

In [5]:
#| hide
from pathlib import Path

In [36]:
#| export
POE_URI = "https://www.pathofexile.com"


class Requester:
    def __init__(self) -> None:
        self.client = requests.session()
        self.client.headers.update({"User-Agent": "POE-Character-Embedder"})

    def get(self, url: str, params: dict = {}, ignore_exception: bool = False) -> dict:
        try:
            response = self.client.get(url=url, params=params)

            if response.status_code != 200:
                if not ignore_exception:
                    raise Exception(
                        f"Error: failed to make successful call to {url} with status code {response.status_code}"
                    )
                else:
                    return {}

        except (
            requests.ConnectTimeout,
            requests.HTTPError,
            requests.ReadTimeout,
            requests.Timeout,
            ConnectionError,
        ):
            self.logger.error(f"Error on get...\n{format_exc()}")

        return response.json()

    def get_character_tree(self, account: str, character: str, **get_kwargs) -> dict:
        # /character-window/get-passive-skills?reqData=0&accountName={account_name}&realm=pc&character={character_name}
        url = POE_URI + f"/character-window/get-passive-skills?reqData=0&accountName={account}&realm=pc&character={character}"
        return self.get(url, **get_kwargs)
    
    def get_character_items(self, account: str, character: str, **get_kwargs) -> dict:
        url=f"{POE_URI}/character-window/get-items"
        params={"accountName": account, "character": character}
        return self.get(url, params=params, **get_kwargs)

In [37]:
example_account = "TheuberClips"
example_character = "UberRankOneOrRiot"

requester = Requester()

example_character_skill_tree = requester.get_character_tree(example_account, example_character)
example_character_items = requester.get_character_items(example_account, example_character)
example_character_skill_tree.keys(), example_character_items.keys()

(dict_keys(['hashes', 'hashes_ex', 'mastery_effects', 'items', 'jewel_data']),
 dict_keys(['items', 'character']))

In [38]:
#| hide
characters_path = Path("../data/raw/Path_of_Exile_HC_SSF_Crucible_league_export.23_Jul_29.06_07_42.0.csv")
characters = pd.read_csv(characters_path)
characters.head()

,Rank,Account,Character,Class,Level,Experience,Dead
0,1,CARNDARAK,cArnTENPERCENT,Slayer,100,4250334444,NaN
1,2,HooWoo,Hoowoo,Champion,100,4250334444,Dead
2,3,cyakimanbo,cyakimanw,Slayer,100,4250334444,Dead
3,4,neradus94,NeraJuggIsWorse,Juggernaut,100,4250334444,Dead
4,5,ryzenier,RyzenierOneStepBehind,Trickster,100,4250334444,NaN


In [39]:
#| hide
characters.shape[0] // 50

20

In [40]:
#| hide
characters_chunk = characters.head(10)
character_chunk_trees = characters_chunk.apply(
    lambda row: requester.get_character_tree(row.Account, row.Character, ignore_exception=True),
    axis=1)
character_chunk_character_items = characters_chunk.apply(
    lambda row: requester.get_character_items(row.Account, row.Character, ignore_exception=True),
    axis=1)


In [41]:
#| hide
characters_chunk_keys = [tuple(l) for l in characters[['Account', 'Character']].values.tolist()]
characters_chunk_keys[:5]

[('CARNDARAK', 'cArnTENPERCENT'),
 ('HooWoo', 'Hoowoo'),
 ('cyakimanbo', 'cyakimanw'),
 ('neradus94', 'NeraJuggIsWorse'),
 ('ryzenier', 'RyzenierOneStepBehind')]

In [42]:
#| hide
character_chunk_dicts = [{"tree": tree, "items": item_dict} for tree, item_dict in zip(character_chunk_trees, character_chunk_character_items)]

In [43]:
#| hide
character_db_path = Path('../data/raw/character_db.json')
assert character_db_path.parent.exists()
character_db = TinyDB(character_db_path)

In [45]:
#| hide
character_chunks = tqdm(np.array_split(characters, characters.shape[0] // 50), ncols=80, leave=False, desc="Getting character data")
for chunk in character_chunks:
    chunk_index = [tuple(l) for l in chunk[['Account', 'Character']].values.tolist()]
    chunk_trees = chunk.apply(
        lambda row: requester.get_character_tree(row.Account, row.Character, ignore_exception=True),
        axis=1)
    chunk_items = chunk.apply(
        lambda row: requester.get_character_items(row.Account, row.Character, ignore_exception=True),
        axis=1)
    chunk_dicts = [
        {
            "account": account, 
            "character": character,
            "tree": tree, 
            "items": item_dict
        } for (account, character), tree, item_dict in zip(
            chunk_index,
            character_chunk_trees, 
            character_chunk_character_items
        )]
    character_db.insert_multiple(chunk_dicts)    

In [47]:
#| hide
import nbdev; nbdev.nbdev_export()